In [ ]:
__author__   = "Federico Ranaldi"

In [ ]:
import numpy as np
import pandas as pd
import string
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df=pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
df["author"].unique()

In [ ]:
df.groupby("author").count()

In [ ]:
#df=df[df["author"].isin(["Mary Wollstonecraft Shelley","Howard Phillips Lovecraft","Edgar Allan Poe"])]
df=df[df["author"].isin(["Mary Wollstonecraft Shelley","Howard Phillips Lovecraft","Edgar Allan Poe","William Shakespeare","Percy Bysshe Shelley"])]

In [ ]:
df.index=range(len(df))

In [ ]:
df.groupby("author").count()

**WORD-TOKENIZATION**

In [ ]:
nltk.download('punkt')

df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row["text"]), axis=1)

**POS-TAGGIN(PART-OF-SPEECH TAGGING)**

In [ ]:

nltk.download('averaged_perceptron_tagger')

df['tokenized_sents'] = df['tokenized_sents'].apply(lambda x: pos_tag(x))

In [ ]:
df['tokenized_sents']

**TO-LOWER-CASE**

In [ ]:
def listOfLists(L):
  newL=[]
  for t in L:
    newL.append(list(t))
  return newL

def toLowerCase(L):
  for l in L:
    l[0]=l[0].lower()
  return L

#trasformazione da lista di tuple a lista di liste
df['tokenized_sents']=df['tokenized_sents'].apply(lambda x: listOfLists(x))

#trasformazione dei token in lower-case
df['tokenized_sents']=df['tokenized_sents'].apply(lambda x: toLowerCase(x))

In [ ]:
df["tokenized_sents"]

**STOPWORDS-REMOVAL**

In [ ]:
nltk.download('stopwords')

stop_list = stopwords.words('english')+list(string.punctuation)+[" "]+[""] #noise removal:insieme alle stopwords viene eliminata la punteggiatura

print(stop_list)

In [ ]:
#RIMOZIONE DELLE STOPWORDS (comprende rimozione dei rumori)
#rimuove troppe parole!!!
#rimuove gli elementi della sottolista b dagli elementi della sottolista a
def removeSublist(a,b):
  for el in b:
    a.remove(el)

#df['tokenized_sents'].apply(lambda x: removeSublist(x,[couple for couple in x if not(set(couple[0]).isdisjoint(stop_list))]))

**NOISE-REMOVAL**

In [ ]:
#solo rimozione di rumori(punteggiatura e spazi vuoti)

def removeSublist(a,b):
  for el in b:
    a.remove(el)

noises_list=list(string.punctuation)+[" "]+[""]

df['tokenized_sents'].apply(lambda x: removeSublist(x,[couple for couple in x if not(set(couple[0]).isdisjoint(noises_list))]))

In [ ]:
df["tokenized_sents"]

**LEMMATIZATION**

In [ ]:
#Lemmatizzazione in base al Pos-Tag
#Attenzione funziona solo con #codice per POS-TAG

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 0

In [ ]:
lemmatizer.lemmatize("going",pos=get_wordnet_pos("VBZ"))

In [ ]:
#Lemmatizzazione in base al Pos-Tag
#Attenzione funziona solo con #codice per POS-TAG

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 0

lemmatizer = WordNetLemmatizer()

def lemmatizeToken(L):
  for i in range(len(L)):
    if(get_wordnet_pos(L[i][1])!=0):
      L[i][0]=lemmatizer.lemmatize(L[i][0],pos=get_wordnet_pos(L[i][1]))
  return L

df["tokenized_sents"]=df['tokenized_sents'].apply(lambda x: lemmatizeToken(x))

In [ ]:
df["tokenized_sents"]

CONCATENAZIONE DELLE COPPIE TOKEN-TAG OPPURE TOKEN-ENTITY

In [ ]:

df["tokenized_sents"]=df["tokenized_sents"].apply(lambda x: ' '.join([el[0]+el[1] for el in x]))

In [ ]:
df.index=range(len(df))

In [ ]:
df.loc[970,"tokenized_sents"]

# **VETTORIZZAZIONE CON TF-IDF**

Il TF-IDF è un algoritmo di vettorizzazione che converte in un formato numerico il nostro corpus facendo emergere termini specifici, pesando diversamente i termini molto rari o molto comuni in modo da assegnare loro un punteggio basso.
TF sta per term frequency, mentre IDF sta per inverse document frequency. 
Il valore TF-IDF legato ad una parola(o token) aumenta proporzionalmente al numero di volte che questa appare nel documento ed è compensato dal numero di documenti nel corpus che la contengono.Se una parola è contenuta in molti documenti allora è probabile che quella non sia una parola altamente specifica per il documento.
Nell'algorimo TF-IDF così come in BOW non viene codificato l'ordine dei token all'interno di un testo e ciò causa un ulteriore perdita di informazione oltre a quella generata dal preprocessing.
Mentre BOW si limita a codificare le parole contenute in un testo preprocessato TFIDF codifica l'informazione che descrive l'importanza di una parola all'interno di un testo che fa parte di un insieme di documenti su cui il modello verrà addestrato.

In [ ]:
# inizializziamo il vettorizzatore
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.95)
# fit_transform applica il TF-IDF ai testi puliti - salviamo la matrice di vettori in X
data_vectorized = vectorizer.fit_transform(df['tokenized_sents'])

In [ ]:
len(vectorizer.get_feature_names_out())

In [ ]:
vectorizer.vocabulary_

In [ ]:
X=data_vectorized.toarray()

In [ ]:
Y=np.empty((len(df),1))
Y

In [ ]:
#trasformazione delle classi targhet da stringhe a numeri
authors=df["author"].unique() #lista/insieme degli autori

targets=np.array(df["author"])

for i in range (len(df)):
  Y[i]=np.where(authors == targets[i])[0][0]

Y=Y.astype("int")
Y

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

In [ ]:
print("Xtrain",type(X_train))
print("Ytrain",type(Y_train))
print("Xtest",type(X_test))
print("Ytest",type(Y_test))
print("Xtrain",X_train.dtype)
print("Ytrain",Y_train.dtype)
print("Xtest",X_test.dtype)
print("Ytest",Y_test.dtype)


**Trasformazione degli array numpy in tensori pytorch**

In [ ]:
import torch

X_train = torch.from_numpy(X_train).float()
Y_train = torch.from_numpy(Y_train).float()

X_test = torch.from_numpy(X_test).float()
Y_test = torch.from_numpy(Y_test).float()

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

# **Creazione della rete neurale feed-forward**

In [ ]:
#il numero di neuroni viene fatto corrispondere al numero di colonne di un vettore rappresentante un dato
#cioè in base al numero di termini che hanno una tfidf maggiore di 0

len(X_train[0])

input_layer_neurons=len(X_train[0])
output_layer_neurons=5

print(input_layer_neurons)
print(output_layer_neurons)

In [ ]:
import torch.nn as nn
import torch.nn.functional as fn

#creazione della classe Net
#Net è una sottoclasse di nn.Module quindi eredita tutti i suoi metodi e attributi
class Net(nn.Module):
  def __init__(self): #costruttore della classe
    super().__init__()  #viene richiamato il costruttore della classe nn.Module

    #definizione dei layers(quindi degli strati della rete che avranno un numero di ingressi e un numero di uscite determinati)
    #nn.linear applica una trasformazione lineare ai dati
    self.fc1=nn.Linear(input_layer_neurons,124)  #Input Layer : ha 28*28 ingressi(quindi 784 neuroni) e 64 uscite possibili(cioé gli ingressi del layer successivo saranno 64)
    self.fc2=nn.Linear(124,64)     #Hidden Layer 1: ha 64 ingressi e 64 uscite
    #self.fc3=nn.Linear(64,64)     #Hidden Layer 2: ha 64 ingressi e 64 uscite
    self.fc4=nn.Linear(64,output_layer_neurons)     #Output Layer : ha 64 ingressi e 10 uscite perché le classi possibili sono 10
  
  #definizione della logica secondo cui i dati attraversano i vari layer
  #ad ogni dato quando attraversa ogni neurone viene applicata la funzione relu
  def forward(self,x):
    x=fn.relu(self.fc1(x))
    x=fn.relu(self.fc2(x))
    #x=fn.relu(self.fc3(x))
    x=self.fc4(x)

    #return x
    return fn.log_softmax(x,dim=1)  #ritorna la distribuzione probabilistica dell'output della rete

In [ ]:
net=Net() #istanziazione della rete neurale

# Caricamento dei dati(comprende suddivisione in batch)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 10

train_data = TensorDataset(X_train,Y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(X_test,Y_test)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# FASE DI TRAINING

In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("gpu available")
else:
  device = torch.device("cpu")

In [ ]:
import torch.optim as optim

#implementazione della loss-function o funzione di costo
#la loss-function calcola quanto gli output della rete neurale si discostano dai valori reali
#il risultato di una loss-function può essere uno scalare(entropia incrociata...) oppure un vettore(one-hot array)

loss_function=nn.CrossEntropyLoss();

#implementazione dell'ottimizzatore 
#l'ottimizzatore si occupa di modificare i pesi dei collegamenti della rete in modo che questa restituisca un output che si avvicini il più possibile al valore corretto
#net.parameters() restituisce un tensore che rappresenta i parametri della rete ossia tutti i pesi della rete
#il learning rate (lr) è un valore che dice di quanto devono essere aggiustati i pesi a ogni iterazione
#Adam è un ottimizzatore che implementa la discesa stocastica del gradiente (Stochastic Gradient Descent)

optimizer=optim.Adam(net.parameters(),lr=0.001)

#l'epoch è rappresenta l'intera fase in cui al modello sono stati passati tutti i dati del trainset suddivisi in batches
#se si effettuano poche epoche il modello non si addestra bene o meglio non aggiusta i propri pesi nel modo migliore
#se si effettuano troppe epoche il modello rischia l'overfitting (si adegua troppo al training set e perde la capacità di generalizzare)

for epoch in range(3):
  for data in train_data:
    X, y = data #suddividiamo l'esempio in features (X) e classe di appartenenza (y)
    #print(X)
    #print(y)
    net.zero_grad() #setta il gradiente a 0
    output=net(X.view(-1,input_layer_neurons))  #ad ogni passo alla rete viene fornito come input l'ouput dell'iterazione precedente
    #print(output)
    loss=fn.nll_loss(output,y.long())  #calcola la loss-function sull'output della rete al passo corrente rispetto al target reale
    loss.backward() #l'errore commesso viene propagato verso ogni collegamento dallo strato di output agli strati più interni(back-propagation)
    optimizer.step()  #basandosi sui valori riportati dal gradiente effettua l'aggiustamento dei parametri della rete in maniera opportuna  
  print(loss) #stampa la loss function dopo ogni epoca

In [ ]:
correct=0 #numero di esempi su cui la rete ha predetto in maniera corretta
total=0 #numero di esempi su cui la rete ha effettuato una predizione

with torch.no_grad(): #nella fase di test non dobbiamo aggiustare i pesi o calcolare la loss function percui non ci interessa il gradiente
  for data in test_data:
    X, y = data
    output=net(X.view(-1,input_layer_neurons))
    #print(y)
    for idx, i in enumerate(output):
      #print(torch.argmax(i),y[idx])
      if(torch.argmax(i)==y[idx]):
        correct+=1
      total+=1

print("ACCURACY: ",round(correct/total,3)*100,"%")

10- 61.19,61.8-----62.8%
3- 77.5